In [5]:
import re
from dataclasses import dataclass
from pydantic import BaseModel
from typing import List, Dict, Optional, Literal

In [110]:
class AnswerOption(BaseModel):
    letter: Literal['А', 'Б', 'В', 'Г']
    option_text: str

class Question(BaseModel):
    id: str
    question: str
    options: List[AnswerOption]
    answer: Literal['А', 'Б', 'В', 'Г']
    chapter : int


In [89]:
with open("/Users/alfa/Code/financial_assistant/data/raw/тест неквал инвестор вопросы.txt", 'r', encoding='utf-8-sig') as f:
    text = f.read()

In [68]:
id2chapter = {
    1: 'Покупка иностранных акций',
    2: 'Акции, не включенные в котировальные списки',
    3: 'Допуск к необеспеченным сделкам (маржинальная торговля)',
    4: 'Заключение договоров РЕПО',
    5: 'Опционы, фьючерсы, производные финансовые инструменты',
    6: 'Структурные облигации',
    7: 'Паи закрытых паевых инвестиционных фондов (ЗПИФ)',
    8: 'Облигации российских эмитентов, которым не присвоен рейтинг или он ниже уровня',
    9: 'Облигации иностранных эмитентов в валюте (еврооблигации) которым не присвоен рейтинг или он ниже нужного уровня',
    10: 'Облигации со структурным доходом',
    11: 'Вопросы для допуска к иностранным ETF'
}

In [69]:
chapter2id = {v:k for k,v in id2chapter.items()}

In [ ]:
def src2questions(text, name, chapter):
    text = text.strip()
    q_num = name.split(':')[1].strip()
    lines = [l for l in text.strip().split('\n') if l.strip() != '']
    question_text = lines[0]
    correct_answer = lines[-1].split(': ')[-1].strip()

    options_text = lines[1:-1]

    options = []
    for opt_text in options_text:
        letter = opt_text[0]
        text = opt_text[3:].strip()
        options.append({
            'letter': letter.strip(),
            'option_text' : text.strip()
        })
        
    question = {
        'id' : q_num,
        'question' : question_text,
        'options' : options,
        'answer' : correct_answer,
        'chapter' : chapter
    }
    return question

In [91]:
chapters_src = re.split(r'Часть \d+\.\s[^\n]+', text)[1:]
chapter_names = re.findall(r'Часть \d+\.\s[^\n]+', text)

In [92]:
questions = []

for src, id_chapter in zip(chapters_src, id2chapter):
    q_src = re.split(r'Вопрос: \d+.\d+', src)[1:]
    q_names = re.findall(r'Вопрос: \d+.\d+', src)
    for qt, qn in zip(q_src, q_names):
        questions.append(src2questions(qt, qn, id_chapter))


In [93]:
questions

[{'id': '1.4',
  'question': 'Ликвидность акции характеризует',
  'options': [{'letter': 'А',
    'option_text': 'Способность инвестора продать акцию с минимальными для него потерями в минимальный срок.'},
   {'letter': 'Б',
    'option_text': 'Разницу цены такой акции на разных торговых площадках.'},
   {'letter': 'В',
    'option_text': 'Вероятность погашения акции компанией – эмитентом.'},
   {'letter': 'Г',
    'option_text': 'Ни один из ответов не является правильным.'}],
  'answer': 'А',
  'chapter': 1},
 {'id': '1.5',
  'question': 'Что из перечисленного не является риском по приобретению акций иностранных эмитентов?',
  'options': [{'letter': 'А',
    'option_text': 'Риск изменения суверенного рейтинга Российской Федерации.'},
   {'letter': 'Б',
    'option_text': 'Риск неожиданного изменения иностранным государством законодательства, регулирующего обращение акций иностранных эмитентов, в неблагоприятную для вас сторону.'},
   {'letter': 'В',
    'option_text': 'Риск обложения 

In [94]:
len(questions)

44

In [113]:
def check_validity(q):
    opt = [AnswerOption(**q['options'][i]) for i in range(4)]
    question = Question(options=opt, **{k:v for k,v in q.items() if k != 'options'})

In [114]:
validity = [check_validity(q) for q in questions]